https://www.opencodez.com/web-development/web-scraping-using-beautiful-soup-part-1.htm

https://stackoverflow.com/questions/54096972/scraping-paginated-results-using-python-beautifulsoup-3

# Importing Packages

In [1]:
import bs4
from bs4 import *
from urllib.request import Request, urlopen
import requests
import pandas
from pandas import DataFrame
import csv

Defining the website URL (chosing a random page from sikayetvar)

In [2]:
url = 'https://www.sikayetvar.com/ulker/gofret?page=3'

We are getting a 403 error so we must mask our browser.

In the code snippet below we will hide our request by sending a HTTP Header. It will look like our request is being sent from Firefox browser.

In [3]:
req = Request(url, headers = {'User-Agent':'Mozilla/5.0'})

In [4]:
webpage = urlopen(req).read()

Basically the code above will send a request and read the webpage 

**We now will use the python library beautiful soup to store and parse through the HTML document of the webpage.**

In [5]:
soup = BeautifulSoup(webpage, 'html.parser')

Before moving on to the scraper we are going to make some exploration about the data we want to exract. We will look at:
    1. Author (the person who wrote the comment)
    2. Title (title of the comment)
    3. Text (the actual comment)

# Authors

In [6]:
soup_author= soup.findAll("article",{"class":["story-card complaint-card","story-card complaint-card solved"]})
len(soup_author)

23

A list of 23 values will be pulled out. From these we will pull the authors of all the comments using the code below

In [7]:
for x in range(23):
 print(soup_author[x].a['title']) 

Süleyman
Ecre
Aziz
Nazan
Sefa
Nurgul
Oğuzhan
Tamer
Timur
Ece
İrem
Nur
Abdullah
Uygar
Mustafa
Ali
Kübra
Duygu
Gülsüm
Yasin
Tarık
Büşra
Şevket


# Titles

In [8]:
soup_title= soup.findAll("h2",{"class":"complaint-title"})
len(soup_title)

23

A list of 23 values will be pulled out. From these we will pull the titles of all the comments using the code below

In [9]:
for x in range(23):
 print(soup_title[x].a['title']) 

Ülker Hayallerle Aldığım Yeni Ürün Mini All Star'da Hüsranla Karşılaştım
Ülker Dido Frambuaz Aromalı
Ülker 9 Kat Tat Diliminde Siyah Leke Var
Ülker Paketi İçerisinde Saç Teli!
Ülker Paket İçinde Çıkan Kokulu Kağıt
Ülker Çikolata İçinden İp Çıkması
Ülker Çikolatalı Gofret İncelmiş
Maalesef Artık Aramızda Ülker Çikolatalı Gofreti Sevmeyen Var
Ülker Paket Mini Gofretin Boş Çıkması!
Ülker Çikolatalı Gofret Sert Çıktı!
Ülker Çikolatalı Gofretin İçindeki Sertlik
Ülker Firmasının Kötü Pazarlama Taktiği!
Ülker Çikolatalı Gofret Gramaj Farkı
Ülker Çikolatalı Ekstra Bitter Gofret
Ülker Sürekli Gramajı Düşürülen Gofret
Ülker Çikolatalı Bitter Gofret Paketinden Yarım Çıktı
Ülker Mini Gofret Açık Paket
Yarım Kaplamalı Ülker Beyaz Çikolatalı Gofret
Ülker Çikolatalı Gofret Ekstradan Plastik Parça Çıktı
Ülker Mini Çikolatalı Gofret Bayat Çıkıyor!
Ülker Çikolatasız Gofret Görmeyen Var Mı
Ülker Çikolatasız Gofret
Ülker Yalandan Beyaz Çikolata


# Comments

In [10]:
soup_comments= soup.findAll("article",{"class":["story-card complaint-card","story-card complaint-card solved"]})
len(soup_comments)

23

A list of 23 values will be pulled out. From these we will pull the comments of all the comments using the code below

In [11]:
for x in range(23):
 print((soup_comments[x]).p.text.strip())

Yeni çıkardığınız 4,5 TL lik 91 gr MINI paketiniz, içerisinde bulunan ürünlerin büyük paketlerine kıyasla çok pahalı. Buna ek olarak içinde 3 tane çikolatalı gofret, 3 tane Albeni, 4 tane metro, 2 tane coco Star var. Allah aşkına üretim planlamada metroyu çok seven mi var coco Star'ı sevmeyen birisi...
Ülker Dido böğürtlen kremalı çikolatadaki şekilsel bozukluk. Gofret tamamen çapraz bir şekilde yerleşmiş. Sanırım hattaki makine arızasından kaynaklanıyor. Ambalaj üzerinde yansıtılan çikolata resmiyle yediğim çikolata arasında bayağı simetrik fark var.
Ülker 9 kat tat Hindistan cevizli kremalı gofretin diliminde sanki yanmış motor yağı dökülmüş gibi karartı çıktı. Ürünü A101'den almıştım. Fişini bulamadığım için iade edemedim. İnternet sitelerinde de hiçbir şekilde şikayet edebilecek sayfa bulamadım.
Ülker çikolatalı gofretten saç teli çıktı. 1 parmak uzunluğuna yakın saç teli ve pakete yapışık şekilde içerisinde. Ülker çikolatalı gofret paketini açtım açar açmaz paketin içerisinde uzun

# Final Product

As the final stage we are going to write a loop containing the pagination and selectors.

Some texts under the h2's are empty so we are going to exclude them in our loop. 

**Scraper**

In [12]:
# Defining urls and htmls using beautifulsoup
root_url = 'https://www.sikayetvar.com/ulker/gofret?page=3'
req = Request(root_url, headers = {'User-Agent':'Mozilla/5.0'})
html = urlopen(req).read()
soup = BeautifulSoup(html, 'html.parser')

# Adjusting Pagination
paging = soup.find("ul",{"class":"pagination"}).find_all("a")
start_page = paging[1].text
last_page = paging[len(paging)-2].text

# Creating a csv file for our scraped data
outfile = open('comments.csv','w', newline='')
writer = csv.writer(outfile)
writer.writerow(["Name", "Title", "Text"])

# Scraper
pages = list(range(1,int(last_page)+1))
for page in pages:
    url = 'https://www.sikayetvar.com/ulker/gofret?page=%s' %(page)
    req = Request(url, headers = {'User-Agent':'Mozilla/5.0'})
    html = urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')

    print ('Processing page: %s' %(page))

    product_name_list = soup.findAll("article",{"class":["story-card complaint-card","story-card complaint-card solved"]})
    for i in product_name_list:
        try: 
            name = i.find("span",{"class":"username"}).text  
            title = i.find("h2", {"class":"complaint-title"}).text
            text = i.find("p").text

            writer.writerow([name, title, text])

        except:
            pass 
outfile.close()
print ('Done')  

Processing page: 1
Processing page: 2
Processing page: 3
Processing page: 4
Processing page: 5
Processing page: 6
Processing page: 7
Processing page: 8
Processing page: 9
Processing page: 10
Processing page: 11
Processing page: 12
Processing page: 13
Done
